In [7]:
from utils import get_polar_from_cartesian
from utils import get_cartesian_from_polar
import torch

In [13]:
theta = torch.full((1,),1.58)   
x = get_cartesian_from_polar(theta=theta,)
print("X", x)
theta_hat = get_polar_from_cartesian(x)
print("Theta", theta_hat)

X tensor([[-0.0092,  1.0000]])
Theta tensor([1.5800])


In [15]:
theta = torch.full((1,),1.58)
phi = torch.full((1,),1.9)   
x = get_cartesian_from_polar(theta = torch.cat([theta, phi]),)
print("X", x)
theta_hat = get_polar_from_cartesian(x)
print("Theta", theta_hat)

X tensor([[-0.0092,  1.0000],
        [-0.3233,  0.9463]])
Theta tensor([1.5800, 1.9000])


In [83]:
import mpmath
ambiant_dim = 3
log_kappa = torch.tensor(0.0)

order = int(ambiant_dim / 2 - 1)
aux_kappa = log_kappa.exp().item()
aux_bessel = float(mpmath.besseli(order, aux_kappa))

log_bessel = torch.tensor(aux_bessel).log()

norm_constant = (log_bessel
+ torch.log(2 * torch.tensor(torch.pi)) * torch.tensor(ambiant_dim / 2)
- log_kappa.item() * order)

print(norm_constant)



tensor(2.9927)


In [84]:

def _eval_poly(y, coef):
    coef = list(coef)
    result = coef.pop()
    while coef:
        result = coef.pop() + y * result
    return result


_I0_COEF_SMALL = [1.0, 3.5156229, 3.0899424, 1.2067492, 0.2659732, 0.360768e-1, 0.45813e-2]
_I0_COEF_LARGE = [0.39894228, 0.1328592e-1, 0.225319e-2, -0.157565e-2, 0.916281e-2,
                  -0.2057706e-1, 0.2635537e-1, -0.1647633e-1, 0.392377e-2]
_I1_COEF_SMALL = [0.5, 0.87890594, 0.51498869, 0.15084934, 0.2658733e-1, 0.301532e-2, 0.32411e-3]
_I1_COEF_LARGE = [0.39894228, -0.3988024e-1, -0.362018e-2, 0.163801e-2, -0.1031555e-1,
                  0.2282967e-1, -0.2895312e-1, 0.1787654e-1, -0.420059e-2]

_COEF_SMALL = [_I0_COEF_SMALL, _I1_COEF_SMALL]
_COEF_LARGE = [_I0_COEF_LARGE, _I1_COEF_LARGE]


def _log_modified_bessel_fn(x, order=0):
    """
    Returns ``log(I_order(x))`` for ``x > 0``,
    where `order` is either 0 or 1.
    """
    assert order == 0 or order == 1

    # compute small solution
    y = (x / 3.75)
    y = y * y
    small = _eval_poly(y, _COEF_SMALL[order])
    if order == 1:
        small = x.abs() * small
    small = small.log()

    # compute large solution
    y = 3.75 / x
    large = x - 0.5 * x.log() + _eval_poly(y, _COEF_LARGE[order]).log()

    result = torch.where(x < 3.75, small, large)
    return result

import math 
print(+ math.log(2 * math.pi) + _log_modified_bessel_fn(log_kappa.exp(), order=0))

tensor(2.0738)


In [81]:
norm_constant_2 =  torch.log(4 * torch.tensor(torch.pi)) + torch.log(torch.sinh(log_kappa.exp())) - log_kappa 

In [82]:
print(norm_constant_2)

tensor(2.6925)


In [72]:
norm_constant_3 = - log_kappa + torch.log(2 * torch.tensor(torch.pi)) + (log_kappa.exp().exp() - torch.exp(-log_kappa.exp()))

In [73]:
print(norm_constant_3)

tensor(4.1883)
